<a href="https://colab.research.google.com/github/mkirby1995/Lambda-SQL-Practice/blob/master/Lambda_School_SQL_Create_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

import psycopg2
import sqlite3
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'psycopg2'

In [3]:
sqlite3.sqlite_version

'3.22.0'

# Write SQL Helper Classes

In [ ]:
class Sqllite_helper:

    def __init__(self):
        self.conn = sqlite3.connect("""/content/lambda_challenge_db.sqlite3""")


    def create_table(self, schema, dataframe, table_name):
        curs = self.conn.cursor()
        curs.execute(schema)

        def force_type(x):
            if type(x) != str:
                return str(x)
            else:
              return x

        values_to_insert = []

        for i in dataframe.values:
            i = i.tolist()
            i = map(force_type, i)
            values_to_insert.append('(' + ', '.join(i) + ')')

        for row in values_to_insert:
            inset_statement = "INSERT INTO " + table_name + ' VALUES' + row

            curs.execute(inset_statement)
        self.conn.commit()



---



# Create DataFrames

## Branch 
- Branch ID
- Location

In [ ]:
branch_ids = ['\"001\"', '\"002\"', '\"003\"', '\"004\"', '\"005\"', '\"006\"', '\"007\"']

locations = ['\"Pacific_NW\"', '\"California\"', '\"West\"', '\"Texas_and_Southwest\"',
             '\"South\"', '\"Midwest_and_Plains\"', '\"Northeast\"']

In [7]:
branch = pd.DataFrame({
                       'branch_id': branch_ids,
                       'location': locations
                      })
branch.head()

,branch_id,location
0,"""001""","""Pacific_NW"""
1,"""002""","""California"""
2,"""003""","""West"""
3,"""004""","""Texas_and_Southwest"""
4,"""005""","""South"""


In [8]:
print(pd.io.sql.get_schema(branch.reset_index(), 'Branch'))

CREATE TABLE "Branch" (
"index" INTEGER,
  "branch_id" TEXT,
  "location" TEXT
)


In [ ]:
branch_schema = """
CREATE TABLE Branch (
    "branch_id" TEXT,
    "location" TEXT,
    PRIMARY KEY (branch_id)
);
"""



---



## Headquarters
- Headquarters ID
- Headquarters City
- Branch ID

In [ ]:
headquarters_ids = ['\"011\"', '\"021\"', '\"031\"', '\"041\"', '\"051\"',
                    '\"061\"', '\"071\"']
                    
headquarters_city = ['\"Seattle\"', '\"Los_Angeles\"', '\"Denver\"',
                     '\"Dallas\"', '\"Atlanta\"', '\"Chicago\"', '\"New_York\"']
# Key branch_ids

In [11]:
headquarters = pd.DataFrame({
                       'headquarters_id': headquarters_ids,
                       'headquarters_city': locations,
                       'branch_id': branch_ids,
                      })
headquarters.head()

,headquarters_id,headquarters_city,branch_id
0,"""011""","""Pacific_NW""","""001"""
1,"""021""","""California""","""002"""
2,"""031""","""West""","""003"""
3,"""041""","""Texas_and_Southwest""","""004"""
4,"""051""","""South""","""005"""


In [12]:
print(pd.io.sql.get_schema(headquarters.reset_index(), 'Headquarters'))

CREATE TABLE "Headquarters" (
"index" INTEGER,
  "headquarters_id" TEXT,
  "headquarters_city" TEXT,
  "branch_id" TEXT
)


In [ ]:
headquarters_schema = """
CREATE TABLE Headquarters (
    "headquarters_id" TEXT,
    "headquarters_city" TEXT,
    "branch_id" TEXT ,
    PRIMARY KEY (headquarters_id),
    FOREIGN KEY (branch_id)
        REFERENCES Branch (branch_id)
);
"""



---



## Order
- Order ID
- Order Date
- Headquarters ID

In [ ]:
from random import randrange
from datetime import timedelta
from datetime import datetime


def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


d1 = datetime.strptime('1/1/2000 1:30 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2018 4:50 AM', '%m/%d/%Y %I:%M %p')

In [ ]:
# 10000 orders
order_ids = list(range(0, 10001))
order_dates = [random_date(d1, d2) for i in range(len(order_ids))]
headquarters_ids = [np.random.choice(headquarters_ids) for i in range(len(order_ids))]

In [ ]:
orders = pd.DataFrame({
                       'order_id': order_ids,
                       'order_date': order_dates,
                       'headquarters_id': headquarters_ids,
                      })

In [17]:
orders['order_date'] = orders['order_date'].astype(str).str[:10]
orders['order_date'] = pd.to_datetime(orders['order_date'], format = "%Y/%m/%d")
orders['order_date'] = orders['order_date'].dt.date

orders.head()

,order_id,order_date,headquarters_id
0,0,2014-09-22,"""051"""
1,1,2010-08-10,"""061"""
2,2,2001-05-02,"""041"""
3,3,2015-12-17,"""071"""
4,4,2012-10-31,"""051"""


In [18]:
print(pd.io.sql.get_schema(orders.reset_index(), 'Order'))

CREATE TABLE "Order" (
"index" INTEGER,
  "order_id" INTEGER,
  "order_date" DATE,
  "headquarters_id" TEXT
)


In [ ]:
orders_schema = """
CREATE TABLE Orders (
    "order_id" INTEGER,
    "order_date" TIMESTAMP,
    "headquarters_id" TEXT ,
    PRIMARY KEY (order_id),
    FOREIGN KEY (headquarters_id)
        REFERENCES Headquarters(headquarters_id) 
);
"""



---



## Order Detail
- Order Detail ID
- Product ID
- Order ID
- Product Quantity

In [ ]:
order_detail_ids = list(range(0, 10001))
products = list(range(0, 301))
product_ids = [np.random.choice(products) for i in range(len(order_detail_ids))] #200 product ID
order_ids = list(range(0, 10001))
product_quantity = [round(np.random.normal(50, 10, 1)[0]) for i in range(len(order_detail_ids))] # random normal distributions

In [21]:
order_details = pd.DataFrame({
                       'order_detail_id': order_detail_ids,
                       'product_id': product_ids,
                       'order_id': order_ids,
                       'product_quantity': product_quantity
                      })
order_details.head()

,order_detail_id,product_id,order_id,product_quantity
0,0,143,0,67.0
1,1,123,1,44.0
2,2,70,2,39.0
3,3,174,3,47.0
4,4,54,4,42.0


In [22]:
print(pd.io.sql.get_schema(order_details.reset_index(), 'Order Detail'))

CREATE TABLE "Order Detail" (
"index" INTEGER,
  "order_detail_id" INTEGER,
  "product_id" INTEGER,
  "order_id" INTEGER,
  "product_quantity" REAL
)


In [ ]:
order_detail_schema = """
CREATE TABLE Order_Detail (
    "order_detail_id" INTEGER ,
    "product_id" INTEGER ,
    "order_id" INTEGER,
    "product_quantity" REAL,
    PRIMARY KEY (order_detail_id),
    FOREIGN KEY (product_id)
        REFERENCES Product(product_id),
    FOREIGN KEY (order_id)
        REFERENCES Orders(order_id)
);
"""



---



## Product
- Product id
- Supplier ID

In [ ]:
suppliers = list(range(0, 40))

products = list(range(0, 301))
supplier_ids = [str(np.random.choice(suppliers)) for i in range(len(products))]

In [25]:
products = pd.DataFrame({
                       'product_id': products,
                       'supplier_id': supplier_ids,
                      })
products.head()

,product_id,supplier_id
0,0,30
1,1,11
2,2,25
3,3,0
4,4,14


In [26]:
print(pd.io.sql.get_schema(products.reset_index(), 'Product'))

CREATE TABLE "Product" (
"index" INTEGER,
  "product_id" INTEGER,
  "supplier_id" TEXT
)


In [ ]:
product_schema = """
CREATE TABLE Product (
    "product_id" INTEGER,
    "supplier_id" TEXT ,
    PRIMARY KEY (product_id),
    FOREIGN KEY (supplier_id)
        REFERENCES Supplier(supplier_id) 
);
"""



---



## Order Detail Delivery
- Delivery id
- Order id
- Order Detail ID

In [ ]:
delivery_ids = list(range(0, 10001))
order_ids = list(range(0, 10001))
order_detail_ids = list(range(0, 10001))

In [29]:
order_detail_delivery = pd.DataFrame({
                       'delivery_id': delivery_ids,
                       'order_id': order_ids,
                       'order_detail id': order_detail_ids
                      })
order_detail_delivery.head()

,delivery_id,order_id,order_detail id
0,0,0,0
1,1,1,1
2,2,2,2
3,3,3,3
4,4,4,4


In [30]:
print(pd.io.sql.get_schema(order_detail_delivery.reset_index(), 'Order Detail Delivery'))

CREATE TABLE "Order Detail Delivery" (
"index" INTEGER,
  "delivery_id" INTEGER,
  "order_id" INTEGER,
  "order_detail id" INTEGER
)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  """Entry point for launching an IPython kernel.


In [ ]:
order_detail_delivery_schema = """
CREATE TABLE Order_Detail_Delivery (
    "delivery_id" INTEGER ,
    "order_id" INTEGER ,
    "order_detail_id" INTEGER,
    PRIMARY KEY (order_detail_id),
    FOREIGN KEY (delivery_id)
        REFERENCES Delivery(delivery_id),
    FOREIGN KEY (order_id)
        REFERENCES Orders(order_id)
);
"""



---



## Delivery
- Delivery ID
- Delivery Date
- Supplier ID

In [ ]:
delivery_ids = list(range(0, 10001))

delivery_dates = orders['order_date']\
    .apply(lambda x:\
           x + timedelta(days= int(round(np.random.normal(10, 3, 1)[0]))))

df = order_details.set_index('product_id').join(products.set_index('product_id'))
df = df.sort_values(by = 'order_id')

supplier_ids = df['supplier_id'].tolist()

In [33]:
delivery = pd.DataFrame({
                       'delivery_id': delivery_ids,
                       'delivery_date': delivery_dates,
                       'supplier_id': supplier_ids
                      })

delivery['delivery_date'] = delivery['delivery_date'].astype(str).str[:10]
delivery['delivery_date'] = pd.to_datetime(delivery['delivery_date'], format = "%Y/%m/%d")
delivery['delivery_date'] = delivery['delivery_date'].dt.date

delivery.head()

,delivery_id,delivery_date,supplier_id
0,0,2014-09-29,35
1,1,2010-08-19,14
2,2,2001-05-08,17
3,3,2015-12-30,18
4,4,2012-11-10,24


In [34]:
print(pd.io.sql.get_schema(delivery.reset_index(), 'Delivery'))

CREATE TABLE "Delivery" (
"index" INTEGER,
  "delivery_id" INTEGER,
  "delivery_date" DATE,
  "supplier_id" TEXT
)


In [ ]:
delivery_schema = """
CREATE TABLE Delivery (
    "delivery_id" INTEGER,
    "delivery_date" TIMESTAMP,
    "supplier_id" TEXT,
    PRIMARY KEY (delivery_id),
    FOREIGN KEY (supplier_id)
        REFERENCES Supplier(supplier_id)
);
"""



---



## Supplier
- Delivery ID
- Delivery Date
- Supplier ID

In [36]:
suppliers = list(set(delivery['supplier_id'].tolist()))
supplier_locations = [np.random.choice(headquarters_city) for i in range(len(suppliers))]

supplier = pd.DataFrame({'supplier_id': suppliers,
                         'location': supplier_locations})

supplier = supplier.sort_values(by = 'supplier_id')
supplier.head()

,supplier_id,location
14,0,"""New_York"""
33,1,"""Atlanta"""
17,10,"""Denver"""
6,11,"""Dallas"""
31,12,"""Atlanta"""


In [37]:
print(pd.io.sql.get_schema(supplier.reset_index(), 'Supplier'))

CREATE TABLE "Supplier" (
"index" INTEGER,
  "supplier_id" TEXT,
  "location" TEXT
)


In [ ]:
supplier_schema = """
CREATE TABLE Supplier (
    "supplier_id" TEXT,
    "location" TEXT,
    PRIMARY KEY (supplier_id)
);
"""



---



# Execute DB Creation

In [39]:
dataframe_list = [branch, headquarters, orders, order_details, products,
                  order_detail_delivery, delivery, supplier]

table_names = ['Branch', 'Headquarters', 'Orders', 'Order_Detail', 'Product',
               'Order_Detail_Delivery', 'Delivery', 'Supplier']

schemas = [branch_schema, headquarters_schema, orders_schema,
           order_detail_schema, product_schema, order_detail_delivery_schema,
           delivery_schema, supplier_schema]

for i in range(len(table_names)):
    print(schemas[i])
    helper = Sqllite_helper()
    helper.create_table(schemas[i], dataframe_list[i], table_names[i])
    print(table_names[i], ' Done')


CREATE TABLE Branch (
    "branch_id" TEXT,
    "location" TEXT,
    PRIMARY KEY (branch_id)
);

Branch  Done

CREATE TABLE Headquarters (
    "headquarters_id" TEXT,
    "headquarters_city" TEXT,
    "branch_id" TEXT ,
    PRIMARY KEY (headquarters_id),
    FOREIGN KEY (branch_id)
        REFERENCES Branch (branch_id)
);

Headquarters  Done

CREATE TABLE Orders (
    "order_id" INTEGER,
    "order_date" TIMESTAMP,
    "headquarters_id" TEXT ,
    PRIMARY KEY (order_id),
    FOREIGN KEY (headquarters_id)
        REFERENCES Headquarters(headquarters_id) 
);

Orders  Done

CREATE TABLE Order_Detail (
    "order_detail_id" INTEGER ,
    "product_id" INTEGER ,
    "order_id" INTEGER,
    "product_quantity" REAL,
    PRIMARY KEY (order_detail_id),
    FOREIGN KEY (product_id)
        REFERENCES Product(product_id),
    FOREIGN KEY (order_id)
        REFERENCES Orders(order_id)
);

Order_Detail  Done

CREATE TABLE Product (
    "product_id" INTEGER,
    "supplier_id" TEXT ,
    PRIMARY KEY 